In [ ]:
!pip install --upgrade pip
!pip install sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
import os


loader = PyPDFLoader("/content/K.Khajabee Resume.pdf")
docs = loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Loaded {len(docs)} docs, split into {len(chunks)} chunks")
# inspect one
print(chunks[0].page_content[:400])

In [ ]:
print("Creating embeddings... this may take a moment.")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
os.environ["MISTRAL_API_KEY"] = "yU8x8V6GtDv2XfYiY6KlDl1yj03IVXrG"

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
rag_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [ ]:
user_query = "What is netflix system recommendation project ?"
response = rag_chain.invoke({"input": user_query})

print("\n--- RESPONSE ---")
print(response["answer"])

In [ ]:
os.environ["MISTRAL_API_KEY"] = "yU8x8V6GtDv2XfYiY6KlDl1yj03IVXrG"